In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline 
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=10,6
import warnings

In [ ]:
data= pd.read_csv('../input/electric-production/Electric_Production.csv')

data['DATE']= pd.to_datetime(data['DATE'], infer_datetime_format=True)
Electricity=data.set_index(['DATE'])

In [ ]:
from datetime import datetime
Electricity.head(5)

In [ ]:
plt.xlabel('Date')
plt.ylabel('no.of consumption')
plt.plot(Electricity)

In [ ]:
rolmean= Electricity.rolling(window=12).mean()
rolstd= Electricity.rolling(window=12).std()
print (rolmean,rolstd)

In [ ]:
orig = plt.plot(Electricity, color= 'orange',label = 'original')
mean= plt.plot(rolmean, color = 'red', label = 'Rolling Mean')
std = plt. plot (rolstd, color = 'Green', label ='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & standard deviation')
plt.show(block = False)

In [ ]:
from statsmodels.tsa.stattools import adfuller

print('Result of dickey fuller test:')
dftest = adfuller(Electricity['Value'], autolag ='AIC')

dfoutput = pd.Series(dftest[0:4], index =['Test Statistic','p-value', '#Lags Used', ' Number  of Observations Used'])
for key,value in dftest [4].items():
    dfoutput['critical Value (%s) '%key] = value
    
print(dfoutput)

In [ ]:
Electricity_log = np.log(Electricity)
plt.plot(Electricity_log)

In [ ]:
MovingAvg = Electricity_log. rolling(window= 12).mean()
MovingSTD = Electricity_log.rolling(window=12).std()
plt.plot(Electricity_log)
plt.plot(MovingAvg , color ='green')

In [ ]:
Deduction = Electricity_log- MovingAvg
Deduction.head(12)

Deduction.dropna(inplace=True)
Deduction.head(10)

In [ ]:
from  statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    MovingAvg = timeseries.rolling(window=12).mean()
    MovingStd = timeseries.rolling(window=12).std()
    
orig = plt.plot(Electricity, color= 'orange',label = 'original')
mean= plt.plot(rolmean, color = 'red', label = 'Rolling Mean')
std = plt. plot (rolstd, color = 'Green', label ='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & standard deviation')
plt.show(block = False)
    
print('Result of dickey fuller test:')
dftest = adfuller(Electricity['Value'], autolag ='AIC')

dfoutput = pd.Series(dftest[0:4], index =['Test Statistic','p-value', '#Lags Used', ' Number  of Observations Used'])
for key,value in dftest [4].items():
    dfoutput['critical Value (%s) '%key] = value
    
print(dfoutput)

In [ ]:
test_stationarity(Deduction)

In [ ]:
exponential = Electricity_log.ewm(halflife=12, min_periods= 0, adjust=True).mean()
plt.plot(Electricity_log)
plt.plot(exponential, color = 'orange')

In [ ]:
datalogScale = Electricity_log - exponential
test_stationarity(datalogScale)

In [ ]:
Diffshifting = Electricity_log - Electricity_log.shift()
plt.plot(Diffshifting )

In [ ]:
Diffshifting.dropna(inplace=True)
test_stationarity(Diffshifting)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(Electricity_logScale)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
 
plt.subplot(411)
plt.plot(Electricity_logScale,  label ='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label ='Trend')
plt.legend(loc ='best')
plt.subplot(413)
plt.plot(seasonal, label ='seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label ='Residuals')
plt.legend(loc='best')
plt.tight_layout()
  

decomposedlogdata = residual
decomposedlogdata.dropna(inplace =True)
test_stationarity(decomposedlogdata)



In [ ]:
decomposeddata = residual
decomposeddata.dropna(inplace =True)
test_stationarity(decomposeddata)

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

lag_acf = acf(Diffshifting, nlags= 20)
lag_pacf = pacf(Diffshifting, nlags= 20, method= 'ols')

plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle= '--', color= 'gray')
plt.axhline(y =-1.96/np.sqrt(len(Diffshifting)), linestyle='--', color= 'gray')
plt.axhline(y =-1.96/np.sqrt(len(Diffshifting)), linestyle='--', color= 'gray')
plt.title('Autocorrelation Function')

plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y= 0,linestyle ='--', color ='gray')
plt.axhline(y =-1.96/np.sqrt(len(Diffshifting)), linestyle='--', color= 'gray')
plt.axhline(y =-1.96/np.sqrt(len(Diffshifting)), linestyle='--', color= 'gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()



In [ ]:
from statsmodels .tsa.arima_model import ARIMA

model= ARIMA(Electricity_log, order=(2,1,2))
results_AR= model.fit(disp =-1)
plt.plot(Diffshifting)
plt.plot(results_AR.fittedvalues, color ='red')
plt.title('RSS: %.4f' %sum((results_AR.fittedvalues- Diffshifting['Value'])**2))
print('Plotting AR model')

In [ ]:
model= ARIMA(Electricity_log, order=(0,1,2))
results_MA= model.fit(disp =-1)
plt.plot(Diffshifting)
plt.plot(results_MA.fittedvalues, color ='red')
plt.title('RSS: %.4f' %sum((results_MA.fittedvalues- Diffshifting['Value'])**2))
print('Plotting AR model')

In [ ]:
model= ARIMA(Electricity_log, order=(2,1,2))
results_ARIMA= model.fit(disp =-1)
plt.plot(Diffshifting)
plt.plot(results_ARIMA.fittedvalues, color ='red')
plt.title('RSS: %.4f' %sum((results_ARIMA.fittedvalues- Diffshifting['Value'])**2))


In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
print (predictions_ARIMA_diff.head())


In [ ]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print(predictions_ARIMA_diff_cumsum.head())

In [ ]:
predictions_ARIMA_log = pd.Series(Electricity['Value'].iloc[0], index= Electricity_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)
predictions_ARIMA_log.head()

In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.figure(figsize=(10,5))
plt.plot(predictions_ARIMA)